# Introduction to Ray AI Runtime (AIR)
---
(*Suggested Time to Complete: 30 minutes*)

✨ Welcome to Part II of "Introduction to Ray"! 🪩

![Map of Ray](images/map.png)

*Figure 1*

Ray AI Runtime (AIR) is a unified set of libraries built on top of Ray for distributed data processing, model training, tuning, model serving, and reinforcement learning, all in Python. AIR provides simple scalable machine learning for individual workloads and end-to-end workflows, bringing together an ever-growing ecosystem of integrations with your favorite machine learning frameworks.

Before we lay out each library and their unique jobs to be done, let's take a moment to motivate Ray AIR by taking a high-level view of the typical data science and machine learning workflow. Developing a machine learning system is an iterative and often cyclical process that touches on the following stages:

1. Data Collection & Feature Engineering: source, sample, and label raw data; preprocess raw data into well-defined input dataset(s)
2. Model Training: the learning part of machine learning that could utilize a popular framework like PyTorch, XGBoost, or Tensorflow
3. Hyperparameter Tuning: improve upon your baseline model by searching a hyperparameter space
3. Model Evaluation: perform batch inference on new data to evaluate perforamnce, potentially triggering more feature engineering or finding a more relevant set of data
4. Deployment: deploy your solution to production and/or serve your model to the end user

Each of the five native libraries that Ray AIR wraps tackles a piece of the ML specific tasks outlined above that you can see illustrated in *Figure 2*. Because this abstraction layer is built on top of Ray Core, it is distributed by nature.

1. 📊 [Ray Data](https://docs.ray.io/en/latest/data/dataset.html): scalable, framework-agnostic loading and transforming raw data across training and prediction
2. 🚂 [Ray Train](https://docs.ray.io/en/latest/train/train.html): distributed multi-node model training with fault tolerance that integrates with your favorite training libraries
3. 📈 [Ray Tune](https://docs.ray.io/en/latest/tune/index.html): scales experiment execution and hyperparameter tuning to optimize model performance
4. 🍦 [Ray Serve](https://docs.ray.io/en/latest/serve/index.html): deploys your model for online inference, with optional microbatching to improve performance
5. 🦾 [Ray RLlib](https://docs.ray.io/en/latest/rllib/index.html): distributed reinforcement learning workloads that integrate with the other Ray AIR libraries above

In this module, we will contextualize Ray Data, Train, Tune, and Serve with a common ML pipeline and discuss how each library facilitates the distinct steps we need to distribute an end-to-end example. Then, we will look at scaling individual workloads with a reinforcement learning specific application for RLlib.

**Learning Objectives**
1. Introduce the high-level data science libraries that compose Ray AIR: Data, Train, Tune, Serve, and RLlib
2. Understand how to use Ray AIR as a unified toolkit to write an end-to-end ML application in Python as well as scale individual jobs
3. Practice key concepts from each stage of the ML pipeline
    - Data - use out-of-the-box `Preprocessor`s to load and transform data
    - Train - use AIR `Trainer`s for supported ML frameworks
    - Tune - use AIR `Tuner`s for hyperparameter search
    - BatchPredictor - use AIR `BatchPredictor` to load model from best checkpoint for batch inference
    - Serve - use `PredictorDeployment` for online inference
    - RLlib - distribute RL workloads with RLlib

**Prerequisites**
- [Introduction to Ray Notebook](https://github.com): introduces Ray as a low-level distributed computing framework and covers key elements of Ray Core

![End to End](images/e2e_air.png)

*Figure 2*

# Predicting Big Tips w/ NYC Taxi Data
***

To illustrate Ray AIR's capabilities, we will walk through an end-to-end example, building a simple machine learning pipeline using Ray Data, Train, Tune, and Serve. Each section will introduce key components, integrations, and typical workloads for the AIR library before demonstrating its functionality with our example application: predicting big tips on yellow taxi cabs in New York City.

Suppose we want to build an application for taxi drivers in NYC that predicts if a given ride will result in a large tip (<20%). This has the potential to influence drivers' decisions when accepting jobs to maximize their margin, and conversations around [information accessbility for gig workers](https://www.nytimes.com/2022/10/11/technology/gig-workers-drivers-para-app.html) are making waves in the news. For this project, let's use the [New York City Taxi & Limousine Commission's Trip Record Data](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page) to build a binary classification model. Starting off, let's take the yellow cab data from June 2021 which contains over 2 million samples with features including `passenger_count`, `trip_distance` (in miles), `fare_amount` (including tax, tip, fees, etc.), `trip_duration` (in seconds), `hour` (hour that trip started), `day_of_week`, and our target `is_big_tip` (whether the tip amount was greater than 20%).

Our workflow will consist of loading data, setting up a preprocessor, training the model with XGBoost, tuning hyperparameters, performing batch inference, and finally serving our online application.

## 1. Ray Data
***
First up, we want to load in the taxi dataset and transform its raw input into features that will be given to our machine learning model.

[Ray Datasets](https://docs.ray.io/en/latest/data/user-guide.html) are the standard way to load and pass data in Ray libraries and applications. This common basis for data handling allows users to leverage different libraries from the Ray ecosystem in whatever way serves their needs without being tethered to a particular framework.

The benefits of using the core `Dataset` abstractions for loading, transforming, and passing references to data in a Ray cluster include:

- **Flexibility**: Compatible with a variety of file formats, data sources, and distributed frameworks, Datasets work seamlessly with library integrations like Dask on Ray and can be passed between Ray tasks and actors without copying data.
- **Performance for ML Workloads**: Datasets offers important features like accelerator support, pipelining, and global random shuffles that accelerate ML training and inference workloads along with basic distributed data transformations such as map, filter, sort, groupby, and repartition.
- **Persistent Preprocessor**: The `Preprocessor` primitive explicitly captures and stores the transformations applied to convert inputs into features and is applied at both training and serving to keep the processing consistent across the pipeline.
- **Built on Ray Core**: inherits scalability to hundreds of nodes, efficient memory usage due to memory across processes on the same node, and object spilling and recovery to handle failures. Because Datasets are just lists of object references, they can be passed between tasks and actors without needing to make a copy of the data, which is crucial for making data-intensive applications and libraries scalable.

In *Figure 3* below, you can see the a general pattern for creating a `Dataset`, configuring a `Preprocessor`, and passing these into the `Trainer` for consistent data handling throughout the pipeline.

![Ray Data Code Snippet](images/data_code.png)

*Figure 3*

Let's take this generic structure and see how it plays out with our tip prediction task.

### 1(a). Import Relevant Packages + Starting Ray
To start, we'll import Ray (check out our [installation instructions](https://docs.ray.io/en/latest/ray-overview/installation.html)) and start a Ray cluster on our machine that can utilize all the cores available to you as workers. We use `ray.is_initialized` to ensure that we only have one Ray cluster active.

In [1]:
import ray

if ray.is_initialized:
    ray.shutdown()

ray.init()

2022-10-24 13:58:13,351	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8266 


Python version:,3.10.6
Ray version:,2.0.0
Dashboard:,http://127.0.0.1:8266


### 1(b). Create Ray Dataset
Here, we read in the data from an S3 `.parquet` datasource, a column-major format designed to support fast data processing.

In [3]:
dataset = ray.data.read_parquet("s3://anyscale-training-data/intro-to-ray-air/nyc_taxi_2021.parquet")

# split data into training and validation subsets
train_dataset, valid_dataset = dataset.train_test_split(test_size=0.3)

# split datasets into blocks for parallel preprocessing
train_dataset.repartition(100)
valid_dataset.repartition(100)

2022-10-24 14:02:45,863	WARNING read_api.py:291 -- ⚠️  The number of blocks in this dataset (1) limits its parallelism to 1 concurrent tasks. This is much less than the number of available CPU slots in the cluster. Use `.repartition(n)` to increase the number of dataset blocks.
Repartition: 100%|██████████| 100/100 [00:00<00:00, 427.26it/s]


Dataset(num_blocks=100, num_rows=811472, schema={passenger_count: double, trip_distance: double, fare_amount: double, trip_duration: int64, hour: int64, day_of_week: int64, is_big_tip: bool, __index_level_0__: int64})

**💻 Coding Excercise 💻**

There exist many [`Dataset` API elements](https://docs.ray.io/en/latest/data/api/dataset.html#) available for common transformations and operations. Using the above as a reference:
1. Inspect the schema from the underlying Parquet metadata.
2. Count how many rows are in the training and validation datasets.
3. Inspect the first five samples of either dataset.
4. What is the average `fare_amount` grouped by `passenger_count`?

In [ ]:
### YOUR CODE HERE ###

### 1(c). Preprocessing
To transform our raw data -> features, we'll define a `Preprocessor`. What's nice about a Ray AIR `Preprocessor` is that it is automatically incorporated...

- **During Training**: `Preprocessor` is passed into a `Trainer` to `fit` and `transform` input `Dataset`s.
- **During Tuning**: each `Trial` will instantiate its own copy of the `Preprocessor` and the fitting and transformation logic will occur once per `Trial`
- **During Checkpointing**: the `Preprocessor` is saved in the `Checkpoint` if was passed into the `Trainer`
- **During Predicting**: if the `Checkpoint` contains a `Preprocessor`, then it will be used to call `transform_batch` on input batches prior to performing inference

In the code below, we define a `MinMaxScaler` preprocessor that will scale the `trip_distance` and `trip_duration` columns by their range.

In [4]:
from ray.data.preprocessors import MinMaxScaler

# create a preprocessor to scale some columns
preprocessor = MinMaxScaler(columns=["trip_distance", "trip_duration"])

**💻 Coding Excercise 💻**

Ray AIR provides several [preprocessors out of the box](https://docs.ray.io/en/latest/ray-air/preprocessors.html#) as well as support for implementing custom preprocessors. 

For this excercise, visualize the distribution for each of the features in our dataset, read through the "Which preprocessor should you use?" section of the linked user guide above, and determine whether `MinMaxScaler` applied to `trip_distance` and `trip_duration` is sufficient.

Later on, you can compare model performance between the given preprocessor and your custom configuration.

In [ ]:
### YOUR CODE HERE ###

**Key Concepts in This Section**

`Dataset`: The standard way to load and exchange data in Ray AIR. In AIR, Datasets are used extensively for data loading, preprocessing, and batch inference.

`Preprocessors`: Preprocessors are primitives that can be used to transform input data into features. Preprocessors operate on Datasets, which makes them scalable and compatible with a variety of datasources and dataframe libraries. A Preprocessor is fitted during Training, and applied at runtime in both Training and Serving on data batches in the same way. AIR comes with a collection of built-in preprocessors, and you can also define your own with simple templates which you can read more about in our [User Guide](https://docs.ray.io/en/latest/ray-air/preprocessors.html).

## 2. Ray Train
***
Following data preprocessing, we can move forward with defining our model for binary classification of big tip rides.

[Ray Train](https://docs.ray.io/en/latest/ray-air/trainer.html) is a library for distributed training on Ray. It offers key tools for different parts of the training workflow, from feature processing, to scalable training, to integrations with ML tracking tools, to export mechanisms for models.

Ray AIR `Trainer`s enable users to distribute training with popular machine learning frameworks like PyTorch, Tensorflow, XGBoost, HuggingFace Transformers, Scikit-Learn, and more. Train supports features like callbacks for early stopping, checkpointing, and integration with Tensorboard, Weights/Biases, and MLflow for observability.

ML pracitioners tend to run into a few common problems with training models that prompt them to consider distributed solutions:

1. training time is too long to be practical
2. the data is too large to fit on one machine
3. the model itself is too large to fit on a single machine

Ray Train tackles the first problem by running distributed multi-node training with fault tolerance, leveraging Ray Data to scale preprocessing and distributed data ingestion. It is also composable with Ray Tune for scaling hyperparameter tuning and outputs the trained model in the form of a `Checkpoint` for batch inference.

In *Figure 4* below, you see that training comes in two major parts: defining the `Trainer` object and then fitting it to the training dataset. In this code snippet, we use a `TorchTrainer`, however, this may be swapped out with any [integrations](https://docs.ray.io/en/latest/ray-air/package-ref.html#trainer-and-predictor-integrations).

![Ray Train Code Snippet](images/train_code.png)

*Figure 4*

Let's put these concepts in practice by applying it to our taxi problem.

### 2(a). Define AIR `Trainer`

There are three broad categories of Trainers that AIR offers:

- Deep Learning Trainers (Pytorch, Tensorflow, Horovod)
- Tree-based Trainers (XGBoost, LightGBM)
- Other ML frameworks (HuggingFace, Scikit-Learn, RLlib)

In the example below, we will use an `XGBoostTrainer`to perform binary classification on these NYC Taxi rides. To construct a `Trainer`, you provide:

- a `ScalingConfig` which specifies how many parallel training workers and what type of resources (CPUs/GPUs) to use per worker during training.
- a collection of datasets and a preprocessor for the provided datasets which configures preprocessing and the datasets to ingest from

Optionally, you can choose to add `resume_from_checkpoint` which is a checkpoint path to resume from, should your training run be interrupted.

Below, we'll set up an `XGBoostTrainer` for our classification task. [XGBoost](https://xgboost.readthedocs.io/en/stable/) is a gradient boosted decision trees library. We'll then supply our `Preprocessor` from the previous step as well as training and validation datasets to ingest.

In [9]:
from ray.air.config import ScalingConfig
from ray.train.xgboost import XGBoostTrainer

trainer = XGBoostTrainer(

    label_column="is_big_tip",
    num_boost_round=100,

    scaling_config=ScalingConfig(
        # number of workers to use
        num_workers=8,
        # whether to use GPU acceleration
        use_gpu=False),

    # XGBoost specific params
    params={
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
        "tree_method": "approx"
    },

    # feed in our datasets and preprocessor
    datasets={"train": train_dataset, "valid": valid_dataset},
    preprocessor=preprocessor
)

### 2(b). Fit the Trainer

To invoke training, call `.fit()`. Trainer objects produce a `Result` object which gives you access to metrics, checkpoints, and errors.

In [10]:
result = trainer.fit()

/Users/emmy/miniforge3/lib/python3.10/site-packages/ray/train/base_trainer.py:343: UserWarning: Executing `.fit()` may leave less than 20% of CPUs in this cluster for Dataset execution, which can lead to resource contention or hangs. To avoid this, reserve at least 20% of node CPUs for Dataset execution by setting `_max_cpu_fraction_per_node = 0.8` in the Trainer scaling_config. See https://docs.ray.io/en/master/data/dataset-internals.html#datasets-and-tune for more info.
  tuner = Tuner(trainable=trainable, run_config=self.run_config)


Trial name,status,loc,iter,total time (s),train-logloss,train-error,valid-logloss
XGBoostTrainer_93eb7_00000,TERMINATED,127.0.0.1:59394,101,17.1879,0.65472,0.384835,0.6583


(_RemoteRayXGBoostActor pid=59424) [14:55:37] task [xgboost.ray]:5107570080 got new rank 4
(_RemoteRayXGBoostActor pid=59422) [14:55:37] task [xgboost.ray]:4827600144 got new rank 2
(_RemoteRayXGBoostActor pid=59425) [14:55:37] task [xgboost.ray]:6212768992 got new rank 5
(_RemoteRayXGBoostActor pid=59426) [14:55:37] task [xgboost.ray]:4978840848 got new rank 7
(_RemoteRayXGBoostActor pid=59423) [14:55:37] task [xgboost.ray]:5101278576 got new rank 3
(_RemoteRayXGBoostActor pid=59427) [14:55:37] task [xgboost.ray]:5121037776 got new rank 6
(_RemoteRayXGBoostActor pid=59421) [14:55:37] task [xgboost.ray]:5089744288 got new rank 0
(_RemoteRayXGBoostActor pid=59420) [14:55:37] task [xgboost.ray]:4829697392 got new rank 1


Result for XGBoostTrainer_93eb7_00000:
  date: 2022-10-24_14-55-38
  done: false
  experiment_id: f3ba88f2e8e34662be0f27851fc2a8e2
  hostname: Emmys-MacBook-Pro-16
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 59394
  time_since_restore: 6.065418004989624
  time_this_iter_s: 6.065418004989624
  time_total_s: 6.065418004989624
  timestamp: 1666648538
  timesteps_since_restore: 0
  train-error: 0.39317630990903824
  train-logloss: 0.677846201610758
  training_iteration: 1
  trial_id: 93eb7_00000
  valid-error: 0.3921651024311375
  valid-logloss: 0.6778842082323272
  warmup_time: 0.002521038055419922
  
Result for XGBoostTrainer_93eb7_00000:
  date: 2022-10-24_14-55-44
  done: false
  experiment_id: f3ba88f2e8e34662be0f27851fc2a8e2
  hostname: Emmys-MacBook-Pro-16
  iterations_since_restore: 44
  node_ip: 127.0.0.1
  pid: 59394
  time_since_restore: 11.403082132339478
  time_this_iter_s: 1.0101821422576904
  time_total_s: 11.403082132339478
  timestamp: 1666648544
  timesteps_

2022-10-24 14:55:50,089	INFO tune.py:758 -- Total run time: 19.33 seconds (19.22 seconds for the tuning loop).


**💻 Coding Excercise 💻**

You can check out the training results from the `Result` object with the following calls:

```python
# returns last saved checkpoing
result.checkpoint

# returns the n best saved checkpoints as configured in `RunConfig.CheckpointConfig`
result.best_checkpoints

# returns the final metrics as reported
result.metrics

# returns the contain an Exception if training failed
result.error
```

Inspect your training result below. What is the reported accuracy for the training and validation runs? Note: `error` is the binary classification error rate in this case calculated as `#(wrong cases)/#(all cases)`

In [ ]:
### YOUR CODE HERE ###

**Key Concepts in This Section**

`Trainer`: Trainers are wrapper classes around third-party training frameworks such as XGBoost, Pytorch, and Tensorflow. They are built to help integrate with core Ray Actors (for distribution), Ray Tune, and Ray Datasets.

## 3. Ray Tune
***
Ray Tune is a Python library for fast hyperparameter tuning at scale. Easily distribute your trial runs to quickly find the best hyperparameters.

**Key Concept**

`Tuner`: Tuners offer scalable hyperparameter tuning as part of Ray Tune. Tuners can work seamlessly with any Trainer but also can support arbitrary training functions.

### Use AIR `Tuner` for Hyperparameter Search
What if you want to do hyperparameter optimization during training and use the best config for the model? Well, you can then use Tuner and supply your training function, `Trainer`, as part of the argument, along with other Tuner configuration.

1. define the hyperparameter space
2. define `TuneConfig` for number of trials and parallelism
3. invoke `tuner.fit()`

![Ray Tune Code Snippet](images/tune_code.png)

*Figure 5*

In [ ]:
from ray import tune

param_space = {"params": {"max_depth": tune.randint(1, 9)}}
metric = "train-logloss"
our_mode="min"

In [ ]:
from ray.tune.tuner import Tuner, TuneConfig
from ray.air.config import RunConfig

tuner = Tuner(
    trainer,
    param_space=param_space,
    tune_config=TuneConfig(num_samples=5, metric=metric, mode=our_mode),
)
# Execute tuning.
result_grid = tuner.fit()

## 4. Ray AIR Checkpoints
***
The AIR trainers, tuners, and custom pretrained models generate Checkpoints. An AIR Checkpoint is a format for models that are used across different components of the Ray AI Runtime. This common format allows easy interoperability among AIR components and seamless integration with external supported machine learning frameworks.

Lost progress during long-running training jobs due to machine failure. The solution is to store the full state of the model periodically, so that partially trained models are available and can be used to resume training from an intermediate point, instead of starting from scratch.

**Key Concept**

`Checkpoints`: The AIR trainers, tuners, and custom pretrained model generate a framework-specific Checkpoint object. Checkpoints are a common interface for models that are used across different AIR components and libraries.

`BatchPredictor`: loads the best model from a checkpoint to perform batch inference

![Batch Predictor Code Snippet](images/batchpredict_code.png)

*Figure 6*

### Use AIR `BatchPredictor` for Batch Prediction
Once you have trained and tuned your model, create a batch prdictor from best model using the `best_result.checkpoint` and do batch inference.

In [ ]:
from ray.train.batch_predictor import BatchPredictor
from ray.train.xgboost import XGBoostPredictor

# I need a best_result

batch_predictor = BatchPredictor.from_checkpoint(result.checkpoint, XGBoostPredictor)

test_dataset = ray.data.read_parquet("data/nyc_taxi_2022.parquet").drop("is_big_tip")

predicted_probabilities = batch_predictor.predict(test_dataset)
print("PREDICTED PROBABILITIES")
predicted_probabilities.show()

## 5. Ray Serve
***
Ray Serve lets you serve machine learning models in real-time or batch using a simple Python API. Serve individual models or create composite model pipelines, where you can independently deploy, update, and scale individual components.

**Key Concept**

`Deployments`: Deploy the model as an inference service by using Ray Serve and the `PredictorDeployment` class.

![Ray Serve Code Snippet](images/serve_code.png)

*Figure 7*

### Use `PredictorDeployment` for Online Inference
Deploy the best model as an inference service by using Ray Serve and the `PredictorDeployment` class. After deploying the service, you can send requests to it.

In [ ]:
from ray import serve
from fastapi import Request
from ray.serve import PredictorDeployment
from ray.serve.http_adapters import pandas_read_json

serve.run(
    PredictorDeployment.options(name="XGBoostService", num_replicas=2, route_prefix="/rayair").bind(
        XGBoostPredictor, result.checkpoint, http_adapter=pandas_read_json
    )
)

In [ ]:
import requests

sample_input = test_dataset.take(1)
sample_input = dict(sample_input[0])

output = requests.post("http://localhost:8000/rayair", json=[sample_input]).json()
print(output)

In [ ]:
ray.shutdown()

## Summary
You've now just created a Ray Dataset, preprocessed some features, built a model with XGBoost, searched a hyperparameter space for the best configuration, loaded the best model from a checkpoint to perform batch inference, and served that model for online inference. Through this end-to-end example, you explored how to use Ray AIR to distribute an entire ML pipeline.

### Key Concepts

- `Datasets`
- `Preprocessors`
- `Trainers`
- `Tuner`
- `Checkpoints`
- `BatchPredictor`
- `Deployments`

### Next Up

Now that you've seen how you can use Ray AIR's unified toolkit to scale an end-to-end machine learning application, let's see how we can use it to scale individual workloads. In the next section we will cover a reinforcement learning example

# Reinforcement Learning on Ray AIR
In this example, we're going to train a reinforcement learning agent using online training. Online training means that the data from the environment is sampled while we are running the algorithm. In contrast, offline training uses data that has been stored on disk before.

## Ray RLLib
Designed for quick interation and a fast path to production, it includes 25+ latest algorithms that are all implemented to run at scale and in multi-agent mode.

RLlib is an open-source library for reinforcement learning (RL), offering support for production-level, highly distributed RL workloads while maintaining unified and simple APIs for a large variety of industry applications. Whether you would like to train your agents in a multi-agent setup, purely from offline (historic) datasets, or using externally connected simulators, RLlib offers a simple solution for each of your decision making needs.

If you either have your problem coded (in Python) as an RL environment or own lots of pre-recorded, historical behavioral data to learn from, you will be up and running in only a few days. RLlib is already used in production by industry leaders in many different verticals such as climate control, industrial control, manufacturing an dlogistics, finance, gaming, automobile, robotics, boat design, and many others.

We can start by running some imports. We're using OpenAI's gym, which is a standard API for reinforcement learning.

In [ ]:
import gym
import numpy

In [ ]:
from ray.air import RunConfig
from ray.air import ScalingConfig
from ray.air import Checkpoint

from ray.train.rl import RLTrainer
from ray.train.rl import RLPredictor

from ray.air import Result

from ray.tune import Tuner

from ray.rllib.algorithms.marwil import BCTrainer

We're going to use the CartPole environment. insert a gif of cartpole as well as description of the premise

In [ ]:
env = gym.make("CartPole-v0")

Set up an RL Trainer??

In [ ]:
trainer = RLTrainer(
    run_config = RunConfig(stop={"training_iteration": 5}),
    scaling_config = ScalingConfig(num_workers=2, use_gpu=False),
    algorithm="PPO",
    config={
        "env": "CartPole-v1",
        "framework": "tf",
    },
)

In [ ]:
def train_rl_ppo_online(num_workers: int, use_gpu: bool = False) -> Result:
    print("Starting online training")
    trainer = RLTrainer(
        run_config=RunConfig(stop={"training_iteration": 5}),
        scaling_config=ScalingConfig(num_workers=num_workers, use_gpu=use_gpu),
        algorithm="PPO",
        config={
            "env": "CartPole-v1",
            "framework": "tf",
        },
    )
    # Todo (krfricke/xwjiang): Enable checkpoint config in RunConfig
    # result = trainer.fit()
    tuner = Tuner(
        trainer,
        _tuner_kwargs={"checkpoint_at_end": True},
    )
    result = tuner.fit()[0]
    return result

In [ ]:
def evaluate_using_checkpoint(checkpoint: Checkpoint, num_episodes) -> list:
    predictor = RLPredictor.from_checkpoint(checkpoint)

    env = gym.make("CartPole-v0")

    rewards = []
    for i in range(num_episodes):
        obs = env.reset()
        reward = 0.0
        done = False
        while not done:
            action = predictor.predict(np.array([obs]))
            obs, r, done, _ = env.step(action[0])
            reward += r
        rewards.append(reward)

    return rewards

In [ ]:
result = train_rl_ppo_online(num_workers=2, use_gpu=False)

In [ ]:
num_eval_episodes = 3

rewards = evaluate_using_checkpoint(result.checkpoint, num_episodes=num_eval_episodes)
print(f"Average reward over {num_eval_episodes} episodes: " f"{np.mean(rewards)}")

### Summary
#### Key Concepts
#### Key API Elements in This Section
#### Next

# Extra Resources
---
If you would like to practice your new skills further with some in-depth examples beyond the embedded coding excercises, take a look at this list of suggested problems:
- Watch the Ray Summit Talk on [Introduction to Ray AIR](https://github.com/ray-project/hackathon5-algo)
- Check out the [Ray AIR Documentation](https://docs.ray.io/en/latest/ray-air/getting-started.html)
- Understand its [Components and APIs](https://docs.ray.io/en/latest/ray-air/package-ref.html)
- Ray AIR [User Guides](https://docs.ray.io/en/latest/ray-air/user-guides.html) and [Examples](https://docs.ray.io/en/latest/ray-air/examples/index.html)


# Next Steps
---
🎉 Congratulations! You have completed the tutorial on an Introduction to Ray AI Runtime! We dicussed each library in Ray AIR (Data, Train, Tune, Serve, RLLib) and saw some example machine learning workloads to be done with each. In the next module, we will introduce the ecosystem of integrated libraries runs on Ray Core's distributed execution engine, and with Ray Clusters, you can deploy your workloads on AWS, GCP, Azure, or on Kubernetes.

From here, you can learn and get more involved with our active community of developers and researchers by checking out the following resources:
- 💻 [Official Ray Website](https://www.ray.io/): Browse the ecosystem and use this site as a hub to get the information that you need to get going and building with Ray.
- 💬 [Join the Community on Slack](https://forms.gle/9TSdDYUgxYs8SA9e8): Find friends to discuss your new learnings in our Slack space.
- 📣 [Use the Discussion Board](https://discuss.ray.io/): Ask questions, follow topics, and view announcements on this community forum.
- 🙋‍♀️ [Join a Meetup Group](https://www.meetup.com/Bay-Area-Ray-Meetup/): Tune in on meet-ups to listen to compelling talks, get to know other users, and meet the team behind Ray.
- 🪲 [Open an Issue](https://github.com/ray-project/ray/issues/new/choose): Ray is constantly evolving to improve developer experience. Submit feature requests, bug-reports, and get help via GitHub issues.